write the cplex code for this,

fill an n x n matrix W with random numbers,

define two variable matrices, A and B, with one being n x r • r x n,

optimize for the best r and entries in A and B such that their product contains an approximation of W using some arbitrary loss

In [208]:
import numpy as np
from docplex.cp.model import CpoModel as Model
from docplex.cp.parameters import CpoParameters

n = 64
r = 4

In [209]:
np.random.seed(0)

# myparams = CpoParameters()
m = Model()

W = np.random.rand(n, n).round(6)

A = np.array(m.integer_var_list(r*n, 0, 1000))
B = np.array(m.integer_var_list(n*r, 0, 1000))

for i in range(n*r):
    m.add_constraint(A[i] * B[i] <= 1_000_000)
    m.add_constraint(A[i] * B[i] >= 0)


A_float = np.array([a/1000 for a in A]).reshape(n, r)
B_float = np.array([b/1000 for b in B]).reshape(r, n)

distance = (( (A_float @ B_float) - (W) )**2).sum()

m.minimize(distance)

In [210]:
solution = m.solve(log_output=True, TimeLimit=300) 
# 0.3136016

CpoSolverException: Solver error: Problem size limit exceeded.
CP Optimizer Community Edition solves problems with search spaces up to 2^1000.
Unrestricted version options (including academia) at https://ibm.co/2s0wqSa


In [ ]:
A_solution = []
B_solution = []

for i in range(n*r):
    A_solution.append(solution[A.tolist()[i]] / 100)
    B_solution.append(solution[B.tolist()[i]] / 100)

In [ ]:
W_hat = np.array(A_solution).reshape(n, r) @ np.array(B_solution).reshape(r, n)

In [199]:

((W_hat - W)**2).sum()


0.6076362299999999

In [200]:
W_hat

array([[0.527 , 0.6544, 0.4704, 0.676 , 0.4647, 0.7898, 0.4788, 0.7174],
       [0.8839, 0.4538, 0.9143, 0.3418, 0.5273, 0.9126, 0.1436, 0.1769],
       [0.1558, 0.8351, 0.6636, 0.9116, 1.0431, 0.7317, 0.3552, 0.8078],
       [0.    , 0.6423, 0.2063, 0.7481, 0.5312, 0.5058, 0.3758, 0.8193],
       [0.4802, 0.6141, 0.1996, 0.678 , 0.4901, 0.4359, 0.904 , 0.7602],
       [0.4808, 0.3643, 0.7013, 0.3063, 0.6266, 0.542 , 0.1106, 0.1183],
       [0.2183, 0.38  , 0.616 , 0.3739, 0.9588, 0.2001, 0.2938, 0.1118],
       [0.6714, 0.2216, 0.2961, 0.1753, 0.2896, 0.2227, 0.645 , 0.1271]])

In [201]:
W

array([[0.5488, 0.7152, 0.6028, 0.5449, 0.4237, 0.6459, 0.4376, 0.8918],
       [0.9637, 0.3834, 0.7917, 0.5289, 0.568 , 0.9256, 0.071 , 0.0871],
       [0.0202, 0.8326, 0.7782, 0.87  , 0.9786, 0.7992, 0.4615, 0.7805],
       [0.1183, 0.6399, 0.1434, 0.9447, 0.5218, 0.4147, 0.2646, 0.7742],
       [0.4562, 0.5684, 0.0188, 0.6176, 0.6121, 0.6169, 0.9437, 0.6818],
       [0.3595, 0.437 , 0.6976, 0.0602, 0.6668, 0.6706, 0.2104, 0.1289],
       [0.3154, 0.3637, 0.5702, 0.4386, 0.9884, 0.102 , 0.2089, 0.1613],
       [0.6531, 0.2533, 0.4663, 0.2444, 0.159 , 0.1104, 0.6563, 0.1382]])